## MiniCaml con liste

Vogliamo estendere l'interprete di MiniCaml per implementare le **liste**, che si comportano come quelle di OCaml.

### Es 1: Albero della sintassi astratta

Estendere l'AST di MiniCaml con un **nuovo tipo** e le seguenti **nuove espressioni**:
1. Espressione per **costruire liste** (composte unicamente con interi, booleani e stringhe):
    - la lista vuota (di un certo tipo) 
    
    __NOTA:__ Ogni lista, anche se vuota, mantiene sempre l'informazione sul tipo di elementi che può contenere. Quindi le espressioni `[int:]` e `[string:]` dovranno essere di tipi diversi...


2. Alcuni **operatori unari** su liste:
    - `cons`, che aggiunga un elemento in testa
    - `is_empty`, che controlla se la lista è vuota
    - `head` e `tail`


3. **Operatori fra liste**:
    - `append`, che concateni due liste

In [2]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound | TList of tname;;

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* Espressioni delle liste *)
    | EmptyList of tname
    | Cons of exp * exp
    | IsEmpty of exp
    | Head of exp
    | Tail of exp
    | Append of exp * exp

type ide = string


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TUnBound
  | TList of tname


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | EmptyList of tname
  | Cons of exp * exp
  | IsEmpty of exp
  | Head of exp
  | Tail of exp
  | Append of exp * exp


### Es 2: Valori esprimibili

Estendere i valori esprimibili e la funzione `getType` per comprendere anche i valori liste.

In [3]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | UnBound
    (* Nuovo valore lista *)
    | MiniList of tname * evT list

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound
    (* Nuovo tipo lista *)
    | MiniList(t, _) -> TList t

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | UnBound
  | MiniList of tname * evT list


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


val getType : evT -> tname = <fun>


### Es 3: Typechecking

Estendere la funzione di typechecking in maniera tale che `typecheck(TList, v)` restituisca `true` se e solo se `v` è un valore di tipo `TList`, cioè è una lista di valori, e tutti gli elementi contenuti in `v` sono del tipo adeguato.

In [4]:
(* Type-checking: true iff v has type t *)
let rec typecheck ((t, v) : (tname * evT)) =
    match t with
    | TInt -> (match v with 
               | Int(_) -> true
               | _ -> false
               )
    | TBool -> (match v with 
                | Bool(_) -> true
                | _ -> false
                )
    | TString -> (match v with
                 | String(_) -> true
                 | _ -> false
                 )
    | TClosure -> (match v with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match v with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    |TUnBound -> (match v with
                 | UnBound -> true
                 | _ -> false
                 )
    | TList(u) -> (match v with
                  | MiniList(u', vals) ->
                      if u <> u' || u' = TUnBound then false else
                          List.for_all (fun v -> typecheck (u, v)) vals
                  | _ -> false
                  );;

val typecheck : tname * evT -> bool = <fun>


In [5]:
let listaVal = MiniList (TInt, [Int(3); Int(5); Bool(true)]);;
typecheck (TList(TInt), listaVal);;

let lista2 = MiniList (TUnBound, [UnBound]);;
typecheck (TList(TUnBound), lista2);;

val listaVal : evT = MiniList (TInt, [Int 3; Int 5; Bool true])


- : bool = false


val lista2 : evT = MiniList (TUnBound, [UnBound])


- : bool = false


### Es 4: Primitive

Implementare le primitive per le liste

In [6]:
(* Errori a runtime *)
exception RuntimeError of string

(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")
    
(* Costrutti per le liste *)
let list_empty t = MiniList (t, []);;

let list_cons (h: evT) (l: evT) =
  let htype = getType h in
    match (typecheck (TList(htype), l), l) with
      | (true, MiniList(htype, vals)) -> MiniList(htype, h::vals)
      | (_, _) -> raise ( RuntimeError "Wrong type");;
    (*if typecheck (TList(htype), l) then
        match l with
          | MiniList(htype, vals) -> MiniList(htype, h::vals)
          | _ -> raise ( RuntimeError "Wrong type")
      else
        raise ( RuntimeError "Wrong type")*)

let list_is_empty (l: evT) =
  match l with
    | MiniList(t, []) -> Bool(true)
    | MiniList(t, vals) -> Bool(false)
    | _ -> raise ( RuntimeError "Wrong type");;

let list_head (l: evT) =
  match l with
    | MiniList(_, []) -> raise ( RuntimeError "Head of empty list")
    | MiniList(_, (v::_)) -> v
    | _ -> raise ( RuntimeError "Wrong type");;

let list_tail (l: evT) =
  match l with
    | MiniList(_, []) -> raise ( RuntimeError "Tail of empty list")
    | MiniList(t, (_::vs)) -> MiniList(t, vs)
    | _ -> raise ( RuntimeError "Wrong type");;

let list_append (l1: evT) (l2: evT) =
  match l1, l2 with
    | MiniList(t1, vals1), MiniList(t2, vals2) ->
      if typecheck (TList(t1), l1) && typecheck (TList(t1), l2) then
        MiniList (t1, vals1 @ vals2)
      else
        raise ( RuntimeError "Wrong type")
    | _, _ -> raise ( RuntimeError "Wrong type");;


exception RuntimeError of string


val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


...

val list_cons : evT -> evT -> evT = <fun>


val list_is_empty : evT -> evT = <fun>


val list_head : evT -> evT = <fun>


val list_tail : evT -> evT = <fun>


val list_append : evT -> evT -> evT = <fun>


### Es 5: Interprete

Completare la funzione eval, facendo pattern matching dei nuovi casi definiti dentro exp, e associandoli alle primitive definite precedentemente.

In [7]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (* Eval delle liste *)
    | EmptyList(t) -> list_empty t
    | Cons(e1, e2) -> list_cons (eval e1 s) (eval e2 s)
    | IsEmpty(e) -> list_is_empty (eval e s)
    | Head(e) -> list_head (eval e s)
    | Tail(e) -> list_tail (eval e s)
    | Append(e1, e2) -> list_append (eval e1 s) (eval e2 s);;

val eval : exp -> evT env -> evT = <fun>


In [14]:
let expr = Letrec("sum", "list",
   IfThenElse(IsEmpty(Den "list"), EInt 0, Sum( Head (Den "list"), Apply(Den "sum", Tail (Den "list")))),
   Apply(Den "sum", Cons(EInt 3, Cons(EInt 10, Cons (EInt (-2), EmptyList (TInt))))));;

eval expr emptyenv;;

val expr : exp =
  Letrec ("sum", "list",
   IfThenElse (IsEmpty (Den "list"), EInt 0,
    Sum (Head (Den "list"), Apply (Den "sum", Tail (Den "list")))),
   Apply (Den "sum",
    Cons (EInt 3, Cons (EInt 10, Cons (EInt (-2), EmptyList TInt)))))


- : evT = Int 11
